In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_07 import *

## Layerwise Sequential Unit Variance (LSUV)

LSUV is a technique developed by Dmitro Mishkin in the paper "All Nou Need is a Good Init", linked below. 
We have seen how fiddly it is to keep a unit variance throughout the layers. If we change the activation or, and we haven't seen this, if we apply dropout, the variance of the layer outputs will change. Even small deviations from unity will turn into exponential deviations. For each case we need a proper initialization, so the idea of the paper is to *learn* it.

LSUV is something we run only once at the beginning of training, not at each batch. We saw how starting with a bad initialization would lead to a large fraction of the weights being zero. This is why we want to start well.

To see how it works, we start as usual getting the MNIST data and a CNN

[Jump_to lesson 11 video](https://course.fast.ai/videos/?lesson=11&t=235)

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

x_train, x_valid = normalize_to(x_train, x_valid)
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

nh, bs = 50, 512
c = y_train.max().item() + 1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [4]:
mnist_view = view_tfm(1, 28, 28)
cbfs = [Recorder,
        partial(AvgStatsCallback, accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

These are the numbers of filters in our ConvNet.

In [5]:
nfs = [8, 16, 32, 64, 64]

We create a new `ConvLayer` class defined as below.

In [6]:
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, ks, padding=ks // 2,
                              stride=stride, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)

    def forward(self, x):
        return self.relu(self.conv(x))

    @property
    def bias(self):
        return -self.relu.sub

    @bias.setter
    def bias(self, v):
        self.relu.sub = -v

    @property
    def weight(self):
        return self.conv.weight

In [7]:
learn, run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

Now we're going to look at the paper [All You Need is a Good Init](https://arxiv.org/pdf/1511.06422.pdf), which introduces *Layer-wise Sequential Unit-Variance* (*LSUV*). We initialize our neural net with the usual technique, then we pass a batch through the model and check the outputs of the linear and convolutional layers. We can then rescale the weights according to the actual variance we observe on the activations, and subtract the mean we observe from the initial bias. That way we will have activations that stay normalized.

We repeat this process until we are satisfied with the mean/variance we observe.

Let's start by looking at a baseline:

In [8]:
run.fit(2, learn)

train: [1.623953125, tensor(0.4441, device='cuda:0')]
valid: [0.537545263671875, tensor(0.8271, device='cuda:0')]
train: [0.2287638671875, tensor(0.9308, device='cuda:0')]
valid: [0.129186474609375, tensor(0.9606, device='cuda:0')]


Now we recreate our model and we'll try again with LSUV. Hopefully, we'll get better results!

In [9]:
learn, run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

Helper function to get one batch of a given dataloader, with the callbacks called to preprocess it.

In [10]:
#export
def get_batch(dl, run):
    run.xb, run.yb = next(iter(dl))
    for cb in run.cbs:
        cb.set_runner(run)
    run('begin_batch')
    return run.xb, run.yb

In [11]:
xb, yb = get_batch(data.train_dl, run)

We only want the outputs of convolutional or linear layers. To find them, we need a recursive function. We can use `sum(list, [])` to concatenate the lists the function finds (`sum` applies the + operate between the elements of the list you pass it, beginning with the initial state in the second argument).

In PyTorch modules can contain modules and so on, so the use of recursion is very common, as in the case below.

In [12]:
#export
def find_modules(m, cond):
    if cond(m):
        return [m]
    return sum([find_modules(o, cond) for o in m.children()], [])


def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
    return isinstance(l, lin_layers)

In [13]:
mods = find_modules(learn.model, lambda o: isinstance(o, ConvLayer))

In [14]:
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ),
 ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 )]

This is a helper function to grab the mean and std of the output of a hooked layer.

In [15]:
def append_stat(hook, mod, inp, outp):
    d = outp.data
    hook.mean, hook.std = d.mean().item(), d.std().item()

In [16]:
mdl = learn.model.cuda()

So now we can look at the mean and std of the conv layers of our model.

In [17]:
with Hooks(mods, append_stat) as hooks:
    mdl(xb)
    for hook in hooks:
        print(hook.mean, hook.std)

0.41918665170669556 0.7644864916801453
0.49116837978363037 0.7047127485275269
0.35965496301651 0.586581826210022
0.3143722116947174 0.5180572867393494
0.23245514929294586 0.31187137961387634


We first adjust the bias terms to make the means 0, then we adjust the standard deviations to make the stds 1 (with a threshold of 1e-3). The `mdl(xb) is not None` clause is just there to pass `xb` through `mdl` and compute all the activations so that the hooks get updated. 

In [18]:
#export
def lsuv_module(m, xb):
    h = Hook(m, append_stat)

    while mdl(xb) is not None and abs(h.mean) > 1e-3:
        m.bias -= h.mean
    while mdl(xb) is not None and abs(h.std-1) > 1e-3:
        m.weight.data /= h.std

    h.remove()
    return h.mean, h.std

We execute that initialization on all the conv layers in order:

In [19]:
for m in mods: print(lsuv_module(m, xb))

(0.1291378289461136, 0.9999998807907104)
(0.12346304208040237, 1.0)
(0.14451444149017334, 1.0)
(0.1671772301197052, 1.0000001192092896)
(0.3147470951080322, 1.0)


Note that the mean doesn't exactly stay at 0. since we change the standard deviation after by scaling the weight.
**Clarify**: how is the above possible? How can dividing by the variance affect the mean?

Then training is beginning on better grounds. **Note** no, it's not, at least in my case. It is extremely unstable, and it returns `NaN` more often than not. **Understand why!**

In [20]:
%time run.fit(2, learn)

train: [0.509774921875, tensor(0.8372, device='cuda:0')]
valid: [0.17039476318359376, tensor(0.9474, device='cuda:0')]
train: [0.11176447265625, tensor(0.9651, device='cuda:0')]
valid: [0.10021398315429687, tensor(0.9691, device='cuda:0')]
CPU times: user 1.54 s, sys: 137 ms, total: 1.68 s
Wall time: 1.68 s


LSUV is particularly useful for more complex and deeper architectures that are hard to initialize to get unit variance at the last layer.

## Export

In [21]:
!python notebook2script.py 07a_lsuv.ipynb

Converted 07a_lsuv.ipynb to exp/nb_07a.py
